In [1]:
import pandas as pd

games_info = pd.read_csv("C:\\Users\\dijan\\Documents\\GitHub\\Reccomender-system-for-board-games\\data\\raw\\games_detailed_info2025.csv")
games_info.rename(columns={'id': 'ID'}, inplace=True)

In [2]:
bgg_reviews = pd.read_csv("C:\\Users\\dijan\\Documents\\GitHub\\Reccomender-system-for-board-games\\data\\raw\\bgg-26m-reviews.csv")

In [3]:
import data_prep
game_features = [ 'yearpublished_bin']
data_preprocessor = data_prep.DataPreprocessor(games_info, bgg_reviews)
game_info_cut = data_preprocessor.discretize_column(games_info, 'usersrated', n_bins=10, percent=0.25, drop_original=True )
game_info_final = data_preprocessor.discretize_column(game_info_cut, 'yearpublished', n_bins=10, drop_original=True)

bgg_reviews.drop(columns=['comment', 'Unnamed: 0', 'name'], inplace=True)
data_preprocessor2 = data_prep.DataPreprocessor(game_info_final, bgg_reviews)
game_merged = data_preprocessor2.merge_datasets(featires_to_add=game_features)

In [7]:
game_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23743150 entries, 0 to 23743149
Data columns (total 4 columns):
 #   Column             Dtype  
---  ------             -----  
 0   user               object 
 1   rating             float64
 2   ID                 int64  
 3   yearpublished_bin  int64  
dtypes: float64(1), int64(2), object(1)
memory usage: 724.6+ MB


In [8]:
import numpy as np

df = game_merged

users = df['user'].unique()
games = df['ID'].unique()
years = df['yearpublished_bin'].unique()

user2idx = {u:i for i, u in enumerate(users)}
game2idx = {g:i for i, g in enumerate(games)}
year2idx = {y:i for i, y in enumerate(years)}

# lista tuple-ova: (user_idx, game_idx, year_bin_idx, rating)
data_entries = [
    (user2idx[u], game2idx[g], year2idx[y], r)
    for u, g, y, r in zip(df['user'], df['ID'], df['yearpublished_bin'], df['rating'])
]



In [9]:
idx2user = {i: u for u, i in user2idx.items()}
idx2game = {i: g for g, i in game2idx.items()}
idx2year = {i: y for y, i in year2idx.items()}


# algoritam 1

In [ ]:
import numpy as np

# Broj korisnika, igara i godina
num_users = len(users)
num_games = len(games)
num_years = len(years)

# Dimenzija latentnog prostora
k = 10  

# Random inicijalizacija latentnih faktora
np.random.seed(42)
U = np.random.normal(scale=0.1, size=(num_users, k))
M = np.random.normal(scale=0.1, size=(num_games, k))
C = np.random.normal(scale=0.1, size=(num_years, k))
S = np.random.normal(scale=0.1, size=(k,))

# Hyperparametri
eta = 0.01  # learning rate
lambda_reg = 0.1  # regularization

# Predikcija
def predict(u_idx, m_idx, c_idx):
    return np.dot(U[u_idx] * M[m_idx] * C[c_idx], S)

# Loss funkcija (squared error + l2 regularization)
def compute_loss(entries):
    loss = 0
    for u_idx, m_idx, c_idx, r in entries:
        pred = predict(u_idx, m_idx, c_idx)
        loss += (r - pred) ** 2
    # Regularizacija
    loss += lambda_reg * (np.sum(U**2) + np.sum(M**2) + np.sum(C**2) + np.sum(S**2))
    return loss

t = 1

for epoch in range(1):
    np.random.shuffle(data_entries)
    for u_idx, m_idx, c_idx, r in data_entries:
        pred = predict(u_idx, m_idx, c_idx)
        err = pred - r  # gradijent za squared error
        
        # Element-wise proizvodi za gradijent
        U_grad = err * (M[m_idx] * C[c_idx] * S) + lambda_reg * U[u_idx]
        M_grad = err * (U[u_idx] * C[c_idx] * S) + lambda_reg * M[m_idx]
        C_grad = err * (U[u_idx] * M[m_idx] * S) + lambda_reg * C[c_idx]
        S_grad = err * (U[u_idx] * M[m_idx] * C[c_idx]) + lambda_reg * S
        
        # Update
        U[u_idx] -= eta * U_grad
        M[m_idx] -= eta * M_grad
        C[c_idx] -= eta * C_grad
        S -= eta * S_grad

    
    if epoch % 1 == 0:
        loss = compute_loss(data_entries)
        print(f"Epoch {epoch+1}, Loss: {loss:.4f}")

Epoch 1, Loss: 1288045594.2523
